In [2]:
import math
from collections import Counter
import numpy as np
import re


In [3]:
def tokenize_sentences(text):
   tokens = re.split(r'(\W+)', text)
   result = []
   for token in tokens:
     if token and token.strip():  #Filters out falsy values such as '', ' ',
       result.append(token.lower())
   return result

In [4]:
def ngrams(words, n):
    ngram_list = []
    
    for i in range(len(words) - n + 1):  
        ngram = tuple(words[i:i+n])
        ngram_list.append(ngram)

    return ngram_list

In [5]:
def brevity_penalty(reference, candidate):
    ref_length = len(reference)
    can_length = len(candidate)

    # Brevity Penalty
    if can_length > ref_length:
        bp = 1
    else:
        penalty = 1 - (ref_length / can_length)
        bp = np.exp(penalty)

    return bp

In [6]:
def clipped_precision(reference, candidate):
    clipped_precision_score = []

    for i in range(1, 5):
        candidate_ngram_counter = Counter(ngrams(candidate, i))  # counts of n-grams for the candidate

        reference_max_counts = Counter()
        for ref in reference:
            ref_ngram_counter = Counter(ngrams(ref, i))  # counts of n-grams for the reference
            reference_max_counts.update(ref_ngram_counter)

        total_ref_ngrams = sum(reference_max_counts.values())

        clipped_counts = 0
        for ngram in candidate_ngram_counter:
            clipped_counts += min(candidate_ngram_counter[ngram], reference_max_counts.get(ngram, 0))

        if total_ref_ngrams > 0:
            precision_i = clipped_counts / total_ref_ngrams
        else:
            precision_i = 0.0

        clipped_precision_score.append(precision_i)

    weights = [0.25] * 4

    s = sum(w_i * math.log(p_i) if p_i > 0 else -math.inf for w_i, p_i in zip(weights, clipped_precision_score))

    bleu = brevity_penalty(reference, candidate) * math.exp(s)
    return bleu


In [47]:

def bleu_score(reference, candidate):
    return clipped_precision(reference, candidate)

ref1 = tokenize_sentences("It is a guide to action that ensures that the military will forever heed Party commands")
ref2 = tokenize_sentences("It is the guiding principle which guarantees the military forces always being under the command of the Party")
ref3 = tokenize_sentences("It is the practical guide for the army always to heed the directions of the party")

# Test Case 1
candidate1 = tokenize_sentences("It is a guide to action which ensures that the military always obeys the commands of the party")
bleu_case1 = bleu_score([ref1,ref2,ref3], candidate1) * 100
print("BLEU Score for Test Case 1:", round(bleu_case1, 1))

# Test Case 2
candidate2 = tokenize_sentences("It is the to action the troops forever hearing the activity guidebook that party direct")
bleu_case2 = bleu_score([ref1,ref2,ref3], candidate2) * 100
print("BLEU Score for Test Case 2:", round(bleu_case2, 1))

BLEU Score for Test Case 1: 53.7
BLEU Score for Test Case 2: 0.0
